## Bibliotecas

In [1]:
import pandas as pd
import spacy as sp
import numpy as np
import logging
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

## Carregando os dados

In [2]:
dados_treino = pd.read_csv('Dados/treino.csv')
dados_teste = pd.read_csv('Dados/teste.csv')

## Pré-processamento com Spacy

In [3]:
# Carregando o modelo de lingua portuguesa.
nlp = sp.load('pt_core_news_sm')

In [4]:
# Criando um gerador de textos para tratamento.
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])

In [5]:
# Criando uma função para tratamento de texto.
def tratatamento_texto(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha # Token é válido se não for stopword e for alfabético
        if e_valido: # Se for token válido, adiciona na lista de tokens válidos.
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2: # Se o texto tiver mais de 2 tokens válidos, retorna o texto tratado.
        return ' '.join(tokens_validos)

In [6]:
texto = 'Um texto qualquer para ser tratado.'
doc = nlp(texto)

# Criando uma lista de textos tratados.
textos_tratados = [tratatamento_texto(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                                batch_size=1000,
                                                                n_process=-1)]

In [7]:
# Criando um dataframe com os textos tratados.
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


## Criando o modelo Word2Vec

In [8]:
# Criando um modelo Word2Vec
w2v_modelo = Word2Vec(sg=0, # CBOW
                      window=2, # Considerando 2 palavras antes e depois.
                      vector_size=300, # Tamanho do vetor do Word2Vec
                      min_count=5, # Considerar apenas palavras que aparecem 5 vezes ou mais. 
                      alpha=0.03, # Taxa de aprendizado.
                      min_alpha=0.007) # Taxa de aprendizado (minima)


### Construindo o vocabulário do modelo

In [9]:
# Imprimindo o tamanho do vocabulário antes do tratamento.
print(len(titulos_tratados))

# Removendo linhas com valores nulos e duplicados.
titulos_tratados = titulos_tratados.dropna().drop_duplicates()

# Imprimindo o tamanho do vocabulário após o tratamento.
print(len(titulos_tratados))

# Criando uma lista de listas de tokens.
lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados['titulo']]

90000
84466


In [10]:
# Configurando o log do modelo Word2Vec.
logging.basicConfig(format='%(asctime)s : - %(message)s', level=logging.INFO)

# Criando um modelo Word2Vec
w2v_modelo = Word2Vec(sg=0, # CBOW
                      window=2, # Considerando 2 palavras antes e depois.
                      vector_size=300, # Tamanho do vetor do Word2Vec
                      min_count=5, # Considerar apenas palavras que aparecem 5 vezes ou mais. 
                      alpha=0.03, # Taxa de aprendizado.
                      min_alpha=0.007) # Taxa de aprendizado (minima)


# Criando o vocabulário do modelo Word2Vec.
w2v_modelo.build_vocab(lista_lista_tokens, # Lista de lista de tokens de nossos títulos.
                       progress_per=5000) # A cada 5000 títulos, imprime uma mensagem de progresso.

2024-01-18 14:53:02,119 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-01-18T14:53:02.119932', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2024-01-18 14:53:02,120 : - collecting all words and their counts
2024-01-18 14:53:02,121 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-01-18 14:53:02,136 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-01-18 14:53:02,145 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-01-18 14:53:02,153 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-01-18 14:53:02,160 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-01-18 14:53:02,170 : - PROGRESS: at sentence #25000, processed 159

### Treinamento CBOW

In [11]:
# Treinando o modelo Word2Vec.
w2v_modelo.train(lista_lista_tokens, # Lista de lista de tokens de nossos títulos.
                 total_examples=w2v_modelo.corpus_count, # Número total de exemplos.
                 epochs=30) # Número de épocas de treinamento.)

2024-01-18 14:53:02,802 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2024-01-18T14:53:02.802709', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2024-01-18 14:53:03,682 : - EPOCH 0: training on 540242 raw words (485990 effective words) took 0.8s, 578190 effective words/s
2024-01-18 14:53:04,322 : - EPOCH 1: training on 540242 raw words (486168 effective words) took 0.6s, 772036 effective words/s
2024-01-18 14:53:05,083 : - EPOCH 2: training on 540242 raw words (486110 effective words) took 0.8s, 647397 effective words/s
2024-01-18 14:53:05,753 : - EPOCH 3: training on 540242 raw words (486069 effective words) took 0.7s, 738953 effective words/s
2024-01-18 14:53:06,385 : - EPOCH 4: training on 540242 raw wo

(14584713, 16207260)

### Treinamento Skip-gram

In [12]:
# Criando um modelo Word2Vec - Skip-gram
w2v_modelo_sg = Word2Vec(sg=1, # Skip-gram
                      window=5, # Considerando 5 palavras antes e depois.
                      vector_size=300, # Tamanho do vetor do Word2Vec
                      min_count=5, # Considerar apenas palavras que aparecem 5 vezes ou mais. 
                      alpha=0.03, # Taxa de aprendizado.
                      min_alpha=0.007) # Taxa de aprendizado (minima)


# Criando o vocabulário do modelo Word2Vec.
w2v_modelo_sg.build_vocab(lista_lista_tokens, # Lista de lista de tokens de nossos títulos.
                       progress_per=5000) # A cada 5000 títulos, imprime uma mensagem de progresso.

# Treinando o modelo Word2Vec.
w2v_modelo_sg.train(lista_lista_tokens, # Lista de lista de tokens de nossos títulos.
                 total_examples=w2v_modelo_sg.corpus_count, # Número total de exemplos.
                 epochs=30) # Número de épocas de treinamento.)

2024-01-18 14:53:22,201 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-01-18T14:53:22.201508', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2024-01-18 14:53:22,202 : - collecting all words and their counts
2024-01-18 14:53:22,203 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-01-18 14:53:22,213 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-01-18 14:53:22,222 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-01-18 14:53:22,230 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-01-18 14:53:22,238 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-01-18 14:53:22,248 : - PROGRESS: at sentence #25000, processed 159

(14585099, 16207260)

## Salvando os modelos

In [13]:
w2v_modelo.wv.save_word2vec_format('Dados/modelo_cbow.txt', binary=False)
w2v_modelo_sg.wv.save_word2vec_format('Dados/modelo_skipgram.txt', binary=False)

2024-01-18 14:54:13,822 : - storing 12924x300 projection weights into Dados/modelo_cbow.txt


2024-01-18 14:54:18,344 : - storing 12924x300 projection weights into Dados/modelo_skipgram.txt


## Criando classificador

In [15]:
# Carregando o modelo Word2Vec - Skip-gram
w2v_modelo_cbow = KeyedVectors.load_word2vec_format('Dados/modelo_cbow.txt')

# Carregando o modelo Word2Vec - Skip-gram
w2v_modelo_sg = KeyedVectors.load_word2vec_format('Dados/modelo_skipgram.txt')

2024-01-18 18:11:11,868 : - loading projection weights from Dados/modelo_cbow.txt
2024-01-18 18:11:17,418 : - KeyedVectors lifecycle event {'msg': 'loaded (12924, 300) matrix of type float32 from Dados/modelo_cbow.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-01-18T18:11:17.341589', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'load_word2vec_format'}
2024-01-18 18:11:17,437 : - loading projection weights from Dados/modelo_skipgram.txt
2024-01-18 18:11:21,703 : - KeyedVectors lifecycle event {'msg': 'loaded (12924, 300) matrix of type float32 from Dados/modelo_skipgram.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-01-18T18:11:21.703816', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'load_word2vec

In [17]:
nlp = sp.load('pt_core_news_sm', disable=['parser', 'ner', 'tagger', 'textcat'])

# Criando uma função para tratamento de texto.
def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha # Token é válido se não for stopword e for alfabético
        if e_valido: # Se for token válido, adiciona na lista de tokens válidos.
            tokens_validos.append(token.text.lower())

    return tokens_validos

texto = 'Um texto qualquer para ser tratado.'
tokens = tokenizador(texto)

In [18]:
# Criando uma função para vetorizar o texto.
def combinacao_de_vetores_por_soma(palavras, modelo):
    # definindo um vetor de tamanho 300 com valores 0.
    vetor_resultante = np.zeros(300)

    # para cada palavra-numero no texto, soma o vetor da palavra ao vetor_resultante.
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)    
        except KeyError:
            pass 
        
    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)

In [19]:
# Criando uma função para vetorizar os dados de treino e teste.
def matriz_vetores(textos, modelo):
    matriz = np.zeros((len(textos), 300))

    for i in range(len(textos)):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

# Vetorizando os dados de treino e teste.
matriz_vetores_treino_cbow = matriz_vetores(dados_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(dados_teste.title, w2v_modelo_cbow)

print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


## Criando modelo de Regressão Logística

In [20]:
# Treinando um modelo de Regressão Logística.
def classificador(modelo, X_treino, X_teste, y_treino, y_teste):
    lr = LogisticRegression(max_iter=800)
    lr.fit(X_treino, y_treino)
    categorias = lr.predict(X_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    return lr

lr_cbow = classificador(w2v_modelo_cbow, matriz_vetores_treino_cbow, matriz_vetores_teste_cbow, dados_treino.category, dados_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.80      0.70     20513
weighted avg       0.82      0.79      0.80     20513

